## Load 'jupyter notebook' from Conda Terminal before beginning to use GPU

In [1]:

from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.optimizers import SGD
import numpy as np
import tensorflow as tf

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print('Tensorflow: ', tf.__version__)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17406735427828061586
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4952306483
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9356040453312676462
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
Tensorflow:  1.11.0


### Load data

In [3]:
X_train = np.load("X_train_split_2.dat")
y_train = np.load("y_train_split_2.dat")

### Shuffle data and reformat for CNN

In [4]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)
# reshape so in form for CNN-Keras
X_train = X_train.reshape(X_train.shape[0], 174, 124, 1)

### Normalize data

In [5]:
X_train = X_train/255

print(X_train.shape)
print(y_train.shape)

(13500, 174, 124, 1)
(13500, 10)


### Split data into training and validation

In [6]:
training_set_size = int(X_train.shape[0] * .80)

# split data into validation set
X_train_validation = X_train[training_set_size:, :,:,:]
y_train_validation = y_train[training_set_size:, :]
y_train = y_train[:training_set_size,:]
X_train = X_train[:training_set_size, :, :, :]
print("Validation x train set:" + str(X_train_validation.shape))
print("X train set:" + str(X_train.shape))
print("Validation y train set:" + str(y_train_validation.shape))
print("Y train set:" + str(y_train.shape))

Validation x train set:(2700, 174, 124, 1)
X train set:(10800, 174, 124, 1)
Validation y train set:(2700, 10)
Y train set:(10800, 10)


## Build the CNN network

In [7]:
# VGG-16 like network from Andrew Ng's course on Coursera

model = Sequential()

# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (7,7), strides=(1,1), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# model.add(Conv2D(32, (7,7), strides=(1,1)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

# model.add(Conv2D(64, (7, 7), strides=(1,1)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# # model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

model.add(Conv2D(64, (7, 7), strides=(1,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.75))
model.add(Dense(10, activation='softmax'))

## Begin training the model

In [8]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

mcp = ModelCheckpoint("models/best_model_2splits_50epochs_highLR", monitor='val_acc', verbose=0, 
                      save_best_only=True, save_weights_only=False, mode='max', period=1)

adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False)

model.compile('adam', 'categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=50 , validation_data=(X_train_validation, y_train_validation), 
         callbacks = [mcp])

Train on 10800 samples, validate on 2700 samples
Epoch 1/50
10800/10800 [==============================] - 28s 3ms/step - loss: 2.4814 - acc: 0.1890 - val_loss: 11.4214 - val_acc: 0.1859
Epoch 2/50
10800/10800 [==============================] - 24s 2ms/step - loss: 2.0943 - acc: 0.2674 - val_loss: 1.8461 - val_acc: 0.3241
Epoch 3/50
10800/10800 [==============================] - 24s 2ms/step - loss: 1.8955 - acc: 0.3180 - val_loss: 1.7988 - val_acc: 0.3100
Epoch 4/50
10800/10800 [==============================] - 24s 2ms/step - loss: 1.8105 - acc: 0.3417 - val_loss: 4.2132 - val_acc: 0.1030
Epoch 5/50
10800/10800 [==============================] - 24s 2ms/step - loss: 1.7225 - acc: 0.3745 - val_loss: 1.5453 - val_acc: 0.4341
Epoch 6/50
10800/10800 [==============================] - 24s 2ms/step - loss: 1.6709 - acc: 0.3914 - val_loss: 1.8053 - val_acc: 0.3689
Epoch 7/50
10800/10800 [==============================] - 24s 2ms/step - loss: 1.6154 - acc: 0.4110 - val_loss: 1.5399 - val_acc

In [13]:
model.save("models/trained_music_classifier.h5")

## Evaluate the model

In [57]:
score = model.evaluate(X_train_validation, y_train_validation, batch_size=32, verbose=1)
print(score)

50/50 [==============================] - 0s 1ms/step
[3.419465198516846, 0.5800000071525574]


### Reset Keras

In [53]:
from keras import backend as K
K.clear_session()

In [ ]:
ann_model = Sequential()

ann_model.add(Flatten())
ann_model.add(Dense(512, activation = 'relu', input_shape = X_input.shape[1:]))
ann_model.add(Dropout(0.5))

ann_model.add(Dense(256), activation = 'relu')
ann_model.add(Dropout(0.5))

ann_model.add(Dense(128), activation = 'relu')
ann_model.add(Dropout(0.5))

ann_model.add(Dense(64), activation = 'relu')
ann_model.add(Dropout(0.5))

ann_model.add(Dense(10), activation = 'softmax')

from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

mcp = ModelCheckpoint("models/best_model_2splits_50epochs_highLR", monitor='val_acc', verbose=0, 
                      save_best_only=True, save_weights_only=False, mode='max', period=1)

adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False)

ann_model.compile(adam, 'categorical_crossentropy',metrics=['accuracy'])

ann_model.fit(X_train, y_train, batch_size=32, epochs=50 , validation_data=(X_train_validation, y_train_validation), 
         callbacks = [mcp])